In [ ]:
# pip install beautifulsoup4
# pip install selenium
# pip install webdriver
# pip install webdriver-manager

In [ ]:
# pip install selenium

In [ ]:
# import selenium
# selenium.__version__

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd


# Replace with your local ChromeDriver path
CHROMEDRIVER_PATH = r"C:\Users\yisha\Desktop\2025-spring-LLM\scraping-reviews-from-googlemaps\Driver\chromedriver-win64\chromedriver.exe"

# Use Service properly (Selenium 4+)
service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service)

In [ ]:
# Function to scrape Google Reviews
review_data = []

def scrape_google_reviews(url):
    driver.get(url)

    # Wait for reviews section to load
    try:
        wait = WebDriverWait(driver, 10)
        reviews_section = wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'm6QErb XiKgde ')]")))
    except Exception as e:
        print("Error: Reviews section not found.", e)
        driver.quit()
        return
    
    body = driver.find_element(By.TAG_NAME, "body")

    for _ in range(3):  # Scroll down a few times to ensure full loading
        body.send_keys(Keys.PAGE_DOWN)
        print("I am scrolling down(range3)")
        time.sleep(2)
        

    # Scroll to load more reviews
    for _ in range(10):  # Adjust based on number of reviews needed
        
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", reviews_section)
        print("I am scrolling down")
        time.sleep(3)  # Allow time for reviews to load

    # Extract reviews
    reviews = driver.find_elements(By.XPATH, "//div[contains(@class, 'jJc9Ad ')]")
    
    for review in reviews:
        try:
            try:
                more_button = review.find_element(By.XPATH, ".//button[contains(@class, 'w8nwRe kyuRq')]")
                driver.execute_script("arguments[0].click();", more_button)
                time.sleep(1)           
            except Exception:
                pass
        
            text = review.find_element(By.XPATH, ".//span[contains(@class, 'wiI7pd')]").text
        
        except Exception as e:
            print("Error extracting review:", e)
            text = "No review text available"
        
        try:
            rating = review.find_element(By.XPATH, ".//span[contains(@class, 'kvMYJc')]").get_attribute("aria-label")
        except:
            rating = "No rating available"


        review_data.append((rating, text))

    # Close driver
    driver.quit()
    
    # print five to check 
    for i in range(5):
        print(review_data[i])

    # save reviews to csv
    df = pd.DataFrame(review_data, columns=['Rating', 'Reviews'])

    df.to_csv('Tiger Suger Reviews-New York-Korean Town.csv', index=False, encoding='utf-8')
    

    
google_maps_url = "https://www.google.com/maps/place/TIGER+SUGAR+(KOREATOWN)/@40.7479287,-73.9903825,17z/data=!3m1!5s0x89c2c22a0528649b:0x56c8962ccf06fc51!4m8!3m7!1s0x89c2597c030838e7:0xe3a3de8bf0d82dd0!8m2!3d40.7479247!4d-73.9878076!9m1!1b1!16s%2Fg%2F11nntd7xsk?entry=ttu&g_ep=EgoyMDI1MDMyNS4xIKXMDSoJLDEwMjExNDU1SAFQAw%3D%3D"
scrape_google_reviews(google_maps_url)




In [ ]:
len(review_data)


In [ ]:
# combine all csv

import pandas as pd
import glob


csv_files = glob.glob("./dataset/*.csv")

df_list = [pd.read_csv(file) for file in csv_files]
# df_list

all_df = pd.concat(df_list, ignore_index=True)

all_df['Rating'] = all_df['Rating'].str.extract(r"(\d+)")

len(all_df)

674

In [ ]:
# Remove no reviews
for i in range(len(all_df)):
    if all_df['Reviews'][i] == 'No review text available':
        all_df = all_df.drop(i)

In [ ]:
all_df.to_csv("All Reviews.csv", index=False)

In [ ]:
all_df.info()

# 674 --> 607 columns

<class 'pandas.core.frame.DataFrame'>
Index: 607 entries, 0 to 652
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   607 non-null    object
 1   Reviews  607 non-null    object
dtypes: object(2)
memory usage: 14.2+ KB
